In [ ]:
# -----------------------------
# 0. Install packages (if needed)
# -----------------------------
# !pip install transformers torch scikit-learn pandas

# -----------------------------
# 1. Imports
# -----------------------------
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader, random_split
import torch.nn as nn

from transformers import BertTokenizer, BertModel
from torch.optim import AdamW


from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score

# 2. Load dataset
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Data/ankasa.csv")
texts = df['content'].astype(str).tolist()
labels = df['label'].astype(int).tolist()

# 3. Tokenizer
MODEL_NAME = 'bert-base-cased'
tokenizer = BertTokenizer.from_pretrained(MODEL_NAME)
MAX_LEN = 128

class NewsDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.float)
        }

# 4. Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    texts, labels, test_size=0.2, random_state=42
)
train_dataset = NewsDataset(X_train, y_train, tokenizer, MAX_LEN)
test_dataset = NewsDataset(X_test, y_test, tokenizer, MAX_LEN)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16)

# 5. Build BERT classifier
class BERTClassifier(nn.Module):
    def __init__(self, model_name):
        super(BERTClassifier, self).__init__()
        self.bert = BertModel.from_pretrained(model_name)
        self.dropout = nn.Dropout(0.3)
        self.linear = nn.Linear(self.bert.config.hidden_size, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        x = self.dropout(pooled_output)
        x = self.linear(x)
        return self.sigmoid(x)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BERTClassifier(MODEL_NAME)
model.to(device)

# -----------------------------
# 6. Optimizer and loss
# -----------------------------
optimizer = AdamW(model.parameters(), lr=2e-5)
criterion = nn.BCELoss()

# -----------------------------
# 7. Training loop
# -----------------------------
EPOCHS = 3

for epoch in range(EPOCHS):
    model.train()
    total_loss = 0
    for batch in train_loader:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device).unsqueeze(1)
        outputs = model(input_ids, attention_mask)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}/{EPOCHS}, Loss: {total_loss/len(train_loader)}")

# -----------------------------
# 8. Evaluation
# -----------------------------
model.eval()
preds, true_labels = [], []

with torch.no_grad():
    for batch in test_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device).unsqueeze(1)
        outputs = model(input_ids, attention_mask)
        preds.extend(torch.round(outputs).cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

accuracy = accuracy_score(true_labels, preds)
precision = precision_score(true_labels, preds)
recall = recall_score(true_labels, preds)

print(f"Test Results - Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Epoch 1/3, Loss: 0.40575951405138855
Epoch 2/3, Loss: 0.1991633489133274
Epoch 3/3, Loss: 0.08809096996679933
Test Results - Accuracy: 0.9241206030150754, Precision: 0.810546875, Recall: 0.8848614072494669


In [ ]:
# Save the model
torch.save(model.state_dict(), "/content/drive/MyDrive/Colab Notebooks/Stored/Bert/bert_fake_news_model.pth")

# Save the tokenizer
tokenizer.save_pretrained("/content/drive/MyDrive/Colab Notebooks/Stored/Bert/bert_fake_news_tokenizer")


('/content/drive/MyDrive/Colab Notebooks/Stored/Bert/bert_fake_news_tokenizer/tokenizer_config.json',
 '/content/drive/MyDrive/Colab Notebooks/Stored/Bert/bert_fake_news_tokenizer/special_tokens_map.json',
 '/content/drive/MyDrive/Colab Notebooks/Stored/Bert/bert_fake_news_tokenizer/vocab.txt',
 '/content/drive/MyDrive/Colab Notebooks/Stored/Bert/bert_fake_news_tokenizer/added_tokens.json')

In [4]:
import os
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from transformers import BertTokenizer, BertModel
from sklearn.metrics import accuracy_score, precision_score, recall_score
from tqdm import tqdm

# --- Config ---
BATCH_SIZE = 8
EPOCHS = 10
MAX_LEN = 100
MODEL_NAME = "bert-base-cased"
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
CHECKPOINT_DIR = "/content/drive/MyDrive/Colab Notebooks/Checkpoints"
os.makedirs(CHECKPOINT_DIR, exist_ok=True)

# --- Load dataset ---
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Data/Binned/ankasa.csv")
texts = df['content'].tolist()
labels = df['label'].astype(int).tolist()

# --- Tokenizer ---
tokenizer = BertTokenizer.from_pretrained(MODEL_NAME)

# --- Dataset class ---
class NewsDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]

        encoding = self.tokenizer(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_tensors="pt"
        )

        return {
            'input_ids': encoding['input_ids'].squeeze(),
            'attention_mask': encoding['attention_mask'].squeeze(),
            'label': torch.tensor(label, dtype=torch.float)
        }

dataset = NewsDataset(texts, labels, tokenizer, MAX_LEN)

# --- Split 80/10/10 ---
train_size = int(0.8 * len(dataset))
val_size = int(0.1 * len(dataset))
test_size = len(dataset) - train_size - val_size
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

# --- Model ---
class DiseaseMythBuster(nn.Module):
    def __init__(self, model_name):
        super(DiseaseMythBuster, self).__init__()
        self.bert = BertModel.from_pretrained(model_name)
        self.dropout = nn.Dropout(0.3)
        self.fc1 = nn.Linear(768, 256)
        self.fc2 = nn.Linear(256, 64)
        self.fc3 = nn.Linear(64, 1)
        self.relu = nn.ReLU()

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        x = self.dropout(self.relu(self.fc1(pooled_output)))
        x = self.dropout(self.relu(self.fc2(x)))
        x = self.dropout(self.fc3(x))
        return x  # logits for BCEWithLogitsLoss

# --- Instantiate model ---
model = DiseaseMythBuster(MODEL_NAME).to(DEVICE)

# --- Multi-GPU support ---
if torch.cuda.device_count() > 1:
    print(f"Using {torch.cuda.device_count()} GPUs")
    model = nn.DataParallel(model)

# --- Class imbalance ---
num_pos = sum(labels)
num_neg = len(labels) - num_pos
pos_weight = torch.tensor([num_neg / num_pos]).to(DEVICE)

criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
optimizer = optim.Adam(model.parameters(), lr=2e-5, betas=(0.9, 0.999), eps=1e-8)

# --- Checkpoint handling ---
start_epoch = 0
checkpoint_file = os.path.join(CHECKPOINT_DIR, "last_checkpoint.pth")
if os.path.exists(checkpoint_file):
    checkpoint = torch.load(checkpoint_file, map_location=DEVICE)
    if isinstance(model, nn.DataParallel):
        model.module.load_state_dict(checkpoint['model_state_dict'])
    else:
        model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    start_epoch = checkpoint['epoch'] + 1
    print(f"Resuming from epoch {start_epoch}")

# --- Mixed precision scaler ---
scaler = torch.cuda.amp.GradScaler()

# --- Training ---
for epoch in range(start_epoch, EPOCHS):
    model.train()
    total_loss = 0
    all_preds, all_labels = [], []

    loop = tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS}")
    for batch in loop:
        input_ids = batch['input_ids'].to(DEVICE)
        attention_mask = batch['attention_mask'].to(DEVICE)
        labels_batch = batch['label'].to(DEVICE).unsqueeze(1)

        optimizer.zero_grad()

        with torch.cuda.amp.autocast():
            outputs = model(input_ids, attention_mask)
            loss = criterion(outputs, labels_batch)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        total_loss += loss.item()
        preds = (torch.sigmoid(outputs) >= 0.5).int().cpu().numpy()
        all_preds.extend(preds)
        all_labels.extend(labels_batch.cpu().numpy())
        loop.set_postfix(loss=loss.item())

    # --- Save checkpoint ---
    state_dict = model.module.state_dict() if isinstance(model, nn.DataParallel) else model.state_dict()
    torch.save({
        'epoch': epoch,
        'model_state_dict': state_dict,
        'optimizer_state_dict': optimizer.state_dict()
    }, checkpoint_file)

    # --- Metrics ---
    acc = accuracy_score(all_labels, all_preds)
    prec = precision_score(all_labels, all_preds, average="binary")
    rec = recall_score(all_labels, all_preds, average="binary")
    print(f"Epoch {epoch+1} - Loss: {total_loss/len(train_loader):.4f}, "
          f"Acc: {acc:.4f}, Prec: {prec:.4f}, Rec: {rec:.4f}")

# --- Validation ---
model.eval()
val_preds, val_labels = [], []
with torch.no_grad():
    for batch in val_loader:
        input_ids = batch['input_ids'].to(DEVICE)
        attention_mask = batch['attention_mask'].to(DEVICE)
        labels_batch = batch['label'].to(DEVICE).unsqueeze(1)

        outputs = model(input_ids, attention_mask)
        preds = (torch.sigmoid(outputs) >= 0.5).int().cpu().numpy()

        val_preds.extend(preds)
        val_labels.extend(labels_batch.cpu().numpy())

val_acc = accuracy_score(val_labels, val_preds)
val_prec = precision_score(val_labels, val_preds, average="binary")
val_rec = recall_score(val_labels, val_preds, average="binary")
print(f"Validation - Acc: {val_acc:.4f}, Prec: {val_prec:.4f}, Rec: {val_rec:.4f}")


/tmp/ipython-input-2522317280.py:119: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()
Epoch 1/10:   0%|          | 0/995 [00:00<?, ?it/s]/tmp/ipython-input-2522317280.py:135: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
Epoch 1/10: 100%|██████████| 995/995 [01:53<00:00,  8.79it/s, loss=0.498]


Epoch 1 - Loss: 0.9241, Acc: 0.5772, Prec: 0.3541, Rec: 0.7971


Epoch 2/10:   0%|          | 0/995 [00:00<?, ?it/s]/tmp/ipython-input-2522317280.py:135: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
Epoch 2/10: 100%|██████████| 995/995 [01:48<00:00,  9.14it/s, loss=0.649]


Epoch 2 - Loss: 0.7791, Acc: 0.6427, Prec: 0.4065, Rec: 0.8690


Epoch 3/10:   0%|          | 0/995 [00:00<?, ?it/s]/tmp/ipython-input-2522317280.py:135: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
Epoch 3/10: 100%|██████████| 995/995 [02:01<00:00,  8.22it/s, loss=0.269]


...

Epoch 8 - Loss: 0.4063, Acc: 0.7585, Prec: 0.5141, Rec: 0.9808


Epoch 9/10:   0%|          | 0/995 [00:00<?, ?it/s]/tmp/ipython-input-2522317280.py:135: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
Epoch 9/10: 100%|██████████| 995/995 [02:00<00:00,  8.29it/s, loss=1.23]


Epoch 9 - Loss: 0.3892, Acc: 0.7570, Prec: 0.5124, Rec: 0.9887


Epoch 10/10:   0%|          | 0/995 [00:00<?, ?it/s]/tmp/ipython-input-2522317280.py:135: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
Epoch 10/10: 100%|██████████| 995/995 [01:58<00:00,  8.39it/s, loss=0.131]


Epoch 10 - Loss: 0.3733, Acc: 0.7683, Prec: 0.5244, Rec: 0.9902
Validation - Acc: 0.9245, Prec: 0.8855, Rec: 0.8040


In [5]:
# Save final model (after training)
final_model_path = os.path.join(CHECKPOINT_DIR, "fake_news_model.pth")
state_dict = model.module.state_dict() if isinstance(model, nn.DataParallel) else model.state_dict()
torch.save(state_dict, final_model_path)
print("Model saved at:", final_model_path)


Model saved at: /content/drive/MyDrive/Colab Notebooks/Checkpoints/fake_news_model.pth


In [4]:
import torch
import torch.nn as nn
from transformers import BertModel, BertTokenizer

# --- Config ---
MODEL_NAME = "bert-base-cased"
final_model_path = "/content/drive/MyDrive/Colab Notebooks/Checkpoints/fake_news_model.pth"

# --- Redefine the model class ---
class DiseaseMythBuster(nn.Module):
    def __init__(self, model_name):
        super(DiseaseMythBuster, self).__init__()
        self.bert = BertModel.from_pretrained(model_name)
        self.dropout = nn.Dropout(0.3)
        self.fc1 = nn.Linear(768, 256)
        self.fc2 = nn.Linear(256, 64)
        self.fc3 = nn.Linear(64, 1)
        self.relu = nn.ReLU()

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        x = self.dropout(self.relu(self.fc1(pooled_output)))
        x = self.dropout(self.relu(self.fc2(x)))
        x = self.dropout(self.fc3(x))
        return x

# --- Rebuild & load trained weights ---
inference_model = DiseaseMythBuster(MODEL_NAME)
inference_model.load_state_dict(torch.load(final_model_path, map_location="cpu"))
inference_model.eval()

print("Model reloaded and ready for inference ✅")


model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Model reloaded and ready for inference ✅


In [2]:
import torch
from transformers import BertTokenizer
import torch.nn.functional as F

# --- Reload tokenizer ---
MODEL_NAME = "bert-base-cased"
tokenizer = BertTokenizer.from_pretrained(MODEL_NAME)

# --- Define prediction function ---
def predict(text, model, tokenizer, max_len=100, device="cpu"):
    model.eval()
    encoding = tokenizer(
        text,
        add_special_tokens=True,
        max_length=max_len,
        padding="max_length",
        truncation=True,
        return_tensors="pt"
    )
    input_ids = encoding["input_ids"].to(device)
    attention_mask = encoding["attention_mask"].to(device)

    with torch.no_grad():
        outputs = model(input_ids, attention_mask)
        probs = torch.sigmoid(outputs).cpu().numpy()[0][0]
        label = int(probs >= 0.5)
        return label, float(probs)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [5]:
sample_text = "gargling salt water vinegar eliminate covid19 coronavirus throat infected persons"
label, confidence = predict(sample_text, inference_model, tokenizer)
print("Prediction:", "Fake" if label == 0 else "Real", "| Confidence:", confidence)


Prediction: Fake | Confidence: 0.02123882621526718
